In [3]:
import tensorflow as tf
import keras

In [4]:
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        
        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)

        b_shape = (output_size,)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)

    def __call__(self, inputs):
        return self.activation(tf.matmul(inputs, self.W) + self.b) #3) this gets called from NaiveSequential layer(x)


    @property
    def weights(self):
        return (self.W, self.b)

In [5]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers # contains instance of dense layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x) #2) calling a dense layer with all inputs, its a dense layer instance calls NaiveDense __call__ to add weithts to inputs and send to an activation fn
        return x

    @property
    def weights(self):
        weights = []
        for layer in self.layers: # iterate over dense layer instances
            weights += layer.weights # for each layer get its weights
        return weights
            

In [6]:
dense1 = NaiveDense(28*28, 512, tf.nn.relu) #initializes 1st hidden layer with initial weights and biases
dense2 = NaiveDense(512, 10, tf.nn.softmax) #initializes 2nd hidden layer with initial weights and biases

model = NaiveSequential([dense1, dense2])

In [7]:
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images)/batch_size)

    def next(self):
        images = self.images[self.index: self.index + self.batch_size]
        labels = self.labels[self.index: self.index + self.batch_size]
        self.index = self.index + self.batch_size
        return images, labels

In [14]:
def one_training_step(model, images_batch, labels_batch):
    # run the fwd pass and compute model's predictions under gradient tape scope
    with tf.GradientTape() as tape:
        predictions = model(images_batch) #1) this calles the NaiveSequential instance __call__ method
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    # compute the gradient of the loss with regard to its weight. 
    # The output gradients is a list where each entry corresponds to a weight from model.weights list.
    # 1-1 mapping between weight and its derivative
    gradients = tape.gradient(average_loss, model.weights) #gets weights and biases

    # update the weights using the gradient
    update_weights(gradients, model.weights)

    return average_loss

In [15]:
LR = 1e-3
from keras import optimizers

optimizer = optimizers.SGD(LR)

def update_weights(gradients, weights):
    # for g, w in zip(gradients, weights):
    #     w.assign_sub(g * LR) # we should never do this by hand (its equivalent of -= for TF variables)
    optimizer.apply_gradients(zip(gradients, weights))
    

In [16]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch in range(epochs):
        print(f"Epoch: {epoch}")
        batch_generator = BatchGenerator(images, labels)
        for batch in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch % 100 == 0:
                print(f"Loss at batch {batch} : {loss: .2f}")

In [17]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28*28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28*28))
test_images = train_images.astype("float32") / 255

In [18]:
fit(model, train_images, train_labels, epochs=5, batch_size=128)

Epoch: 0
Loss at batch 0 :  5.58
Loss at batch 100 :  2.22
Loss at batch 200 :  2.19
Loss at batch 300 :  2.09
Loss at batch 400 :  2.24
Epoch: 1
Loss at batch 0 :  1.90
Loss at batch 100 :  1.86
Loss at batch 200 :  1.81
Loss at batch 300 :  1.71
Loss at batch 400 :  1.85
Epoch: 2
Loss at batch 0 :  1.58
Loss at batch 100 :  1.56
Loss at batch 200 :  1.48
Loss at batch 300 :  1.42
Loss at batch 400 :  1.52
Epoch: 3
Loss at batch 0 :  1.32
Loss at batch 100 :  1.33
Loss at batch 200 :  1.22
Loss at batch 300 :  1.20
Loss at batch 400 :  1.27
Epoch: 4
Loss at batch 0 :  1.12
Loss at batch 100 :  1.15
Loss at batch 200 :  1.03
Loss at batch 300 :  1.05
Loss at batch 400 :  1.11
